# Chapter 11: Market Risk


In this chapter, we will introduce the concept of market risk, focusing on the potential losses arising from unusual market price fluctuations. We will explore various approaches to measuring market risk, including the parametric RiskMetrics model, the nonparametric historical (back) simulation model, the Monte Carlo simulation model, and the method of quantifying expected shortfall (ES).

Chapter Objectives:

1. Understand market risk.
2. Learn about the RiskMetrics model.
3. Explore historical (back) simulations.
4. Examine the Monte Carlo simulation.
5. Understand and quantify expected shortfall

In [1]:
from scipy.stats import norm

equity=4000000
mean=0
std=0.02
dear_stk=equity*std*norm.ppf(0.99)
print(f"Daily value at risk of the stock investment is ${dear_stk:.2f}")


Daily value at risk of the stock investment is $186107.83


In [2]:
bond=5000000
y=0.05
duration=10
mean_y=0
std_y=0.0015
yield_chg=std_y*norm.ppf(0.99)
mod_dur=duration/(1+y)
dear_b=bond*mod_dur*yield_chg
print("The adverse change in bond yield is", round(yield_chg,4))
print(f"Daily value at risk of the bond investment is ${dear_b:.2f}")


The adverse change in bond yield is 0.0035
Daily value at risk of the bond investment is $166167.71


In [3]:
pound=10000000
fx=1.25
mean_fx=0
std_fx=0.0025
fx_chg=std_fx*norm.ppf(0.99)
dollars=pound*fx
dear_fx=dollars*fx_chg
print(f"The value of the FX position is ${dollars:.2f}")
print(f"Daily value at risk of the currency investment is ${dear_fx:.2f}")


The value of the FX position is $12500000.00
Daily value at risk of the currency investment is $72698.37


In [4]:
import numpy as np

corr12=0.4
corr13=0.1
corr23=-0.2
dear_p = np.sqrt(dear_stk**2 + dear_b**2 + dear_fx**2+ \
                 2*corr12*dear_stk*dear_b\
+ 2*corr13*dear_stk*dear_fx + 2*corr23*dear_b*dear_fx)
print(f"DEAR for the portfolio is ${dear_p:.2f}")


DEAR for the portfolio is $300244.69


In [5]:
MU = np.matrix([[dear_stk,
                dear_b,
                dear_fx]])  
SIGMA = np.matrix([[1, 0.4, 0.1],
                   [0.4, 1, -0.2],
                   [0.1, -0.2, 1]])
DEAR_P_MATRIX = np.sqrt(MU*SIGMA*MU.T).item(0,0)
print(f"DEAR for the portfolio is ${DEAR_P_MATRIX:.2f}")


DEAR for the portfolio is $300244.69


In [6]:
!pip install pandas_datareader yfinance
from datetime import date, timedelta
from pandas_datareader import data as pdr
import yfinance
yfinance.pdr_override()

# Get the ticker symbol
ticker = 'MSFT' # 1
# Set the start and end dates
end_date = date.today().strftime("%Y-%m-%d")  # 2
start_date = (date.today() - timedelta(days=365*5)).strftime("%Y-%m-%d")
# Retieve prices
stock = pdr.get_data_yahoo(ticker, start = start_date, end = end_date) # 3
# Calculate returns
stock['ret_stock'] = (stock['Adj Close']/stock['Adj Close'].shift(1))-1 # 4
# Get the one percentile value
VaR = stock['ret_stock'].quantile([0.01]) # 5
print("the VaR for MSFT is", round(VaR.iloc[0],5)) # 6


[*********************100%***********************]  1 of 1 completed
the VaR for MSFT is -0.04541


C:\Users\hlliu2\Anaconda3\envs\MLA\lib\site-packages\yfinance\utils.py:768: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [7]:
# Use the ticker symbol of S&P500
ticker = '^GSPC'
# Set the start and end dates
end_date = date.today().strftime("%Y-%m-%d")
start_date = (date.today() - timedelta(days=365*5)).strftime("%Y-%m-%d")
# Retieve prices
stock = pdr.get_data_yahoo(ticker, start = start_date, end = end_date)
# Calculate returns
stock['ret_stock'] = (stock['Adj Close']/stock['Adj Close'].shift(1))-1
# Get the one percentile value
VaR = stock['ret_stock'].quantile([0.01])
print("the VaR for S&P500 is", round(VaR.iloc[0],5))


[*********************100%***********************]  1 of 1 completed
the VaR for S&P500 is -0.03594


C:\Users\hlliu2\Anaconda3\envs\MLA\lib\site-packages\yfinance\utils.py:768: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [8]:
# import numpy and generate two numpy arrays
import numpy as np

ret_mkt = np.zeros((10000,)) # 1
ret_p = np.zeros((10000,))

for i in range(len(ret_mkt)): # 2
    ret_mkt[i] = np.random.normal(0.0002, 0.005, 1)[0] # 3
    ret_p[i] = 0.0001 + 1.2 * ret_mkt[i] # 4
   
# Find the one percentile of the value of ret_p
one_pct = np.percentile(ret_p, 1) # 5
print("the VaR on the portfolio is", one_pct)


the VaR on the portfolio is -0.01327048830515097


In [9]:
import numpy as np
import scipy.stats as stats
# Define parameters for the normal distribution (mean and standard deviation)
mu = 0
sigma = 1
# Define the number of intervals within the 99th to 100th percentile
n_intervals = 999
# Generate percentile values from 99.001% to 99.999%
percentiles = np.linspace(99.001, 99.999, n_intervals)
# Calculate VaR at each percentile
vars_at_percentiles = stats.norm.ppf(percentiles / 100, mu, sigma)
# Calculate the average of these VaR values to estimate ES
estimated_es = np.mean(vars_at_percentiles)
print(f"Estimated Expected Shortfall (ES): {estimated_es:.2f}")


Estimated Expected Shortfall (ES): 2.66


In [10]:
from scipy.stats import norm # 1

std = 0.02 # 2

VaR = - std*norm.ppf(0.99)  # 3
print("the value at risk is", round(VaR,5))

expected_shortfalls = [] # 4

for i in range(1000):  # 5
    confidence = 0.99+i*0.00001  # 6
    loss = -std * norm.ppf(confidence)
    expected_shortfalls.append(loss)
   
expected_shortfall = sum(expected_shortfalls)/len(expected_shortfalls) # 7
print("the expected shortfall is", round(expected_shortfall,5))


the value at risk is -0.04653
the expected shortfall is -0.05328


In [11]:
import scipy.stats as stats
# Given data
mu = 0
sigma = 0.02
p = 0.99

# Calculate the z-score for the 99% percentile
z_p = stats.norm.ppf(p)

# Calculate the probability density at the z-score
phi_z_p = stats.norm.pdf(z_p)

# Calculate the Expected Shortfall for the distribution
ES = mu + sigma * phi_z_p / (1 - p)
print(f"Estimated Expected Shortfall (ES): {ES:.4f}")


Estimated Expected Shortfall (ES): 0.0533


In [12]:
payoff_a = {100:0.6, 50:0.39, -100:0.01}
for payoff, p in payoff_a.items():
    if p==0.01:
        es_a = payoff
print("Expected shortfall for distribution a is", es_a,"million")
       
payoff_b = {100:0.6, 50:0.39, -60:0.0025, -100:0.0075}
avg_b=0
p_b=0
for payoff, p in payoff_b.items():
    if p==0.0025 or p==0.0075:
        avg_b += payoff*p
        p_b += p
es_b=avg_b/p_b
print("Expected shortfall for distribution b is", es_b,"million")


Expected shortfall for distribution a is -100 million
Expected shortfall for distribution b is -90.0 million
